In [2]:
import time
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import pickle
from skimage.transform import resize
from skimage import feature
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
image_lst = []
label_lst = []
folder_path = "Sign"
items = os.listdir(folder_path)
folders = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]
for folder in folders:
    sign_path = os.path.join(folder_path,folder)
    sign_items = os.listdir(sign_path)
    # Filter out only the files (not directories)
    img_dir = [item for item in sign_items if os.path.isfile(os.path.join(sign_path, item))]
    for img_path in img_dir:
        img_p = os.path.join(sign_path,img_path)
        img = cv2.imread(img_p)
        image_lst.append(img)
        label_lst.append(folder)


In [12]:
print('Number of objects: ',len(image_lst))
print('class names: ',list(set(label_lst)))

Number of objects:  653
class names:  ['priority', 'hight_way', 'cross_way', 'roundabout', 'forbit', 'parking', 'stop', 'traight', 'exit_hight_way']


In [3]:
def preprocess_img(img):
  if len(img.shape)>2:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = img.astype(np.float32)

  resized_img = resize(
      img,
      output_shape=(64,64),
      anti_aliasing=True
  )
  hog_feature = feature.hog(
      resized_img,
      orientations=9,
      pixels_per_cell=(8,8),
      cells_per_block=(2,2),
      block_norm="L2",
      feature_vector=True
  )
  return hog_feature

In [14]:
img_feature_lst = []
for img in image_lst:
  hog_feature = preprocess_img(img)
  img_feature_lst.append(hog_feature)
img_features = np.array(img_feature_lst)
print("X shape: ")
print(img_features.shape)

X shape: 
(653, 1764)


In [15]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_lst)


In [16]:
print(label_encoder.classes_)

['cross_way' 'exit_hight_way' 'forbit' 'hight_way' 'parking' 'priority'
 'roundabout' 'stop' 'traight']


In [17]:
random_state = 0
test_size = 0.3
is_shuffle = True
X_train, X_val, y_train, y_val = train_test_split(
    img_features,encoded_labels,
    test_size = test_size,
    random_state = random_state,
    shuffle = is_shuffle
)

In [18]:
X_train

array([[0.01199425, 0.00367688, 0.00120082, ..., 0.01889051, 0.00339817,
        0.00722683],
       [0.13502465, 0.12259669, 0.08772346, ..., 0.00919772, 0.00413064,
        0.00759265],
       [0.03895593, 0.00091717, 0.00049443, ..., 0.00156643, 0.00060468,
        0.00116945],
       ...,
       [0.08035994, 0.01908555, 0.01794297, ..., 0.06035848, 0.06166307,
        0.04117857],
       [0.05432067, 0.        , 0.00120186, ..., 0.00559256, 0.        ,
        0.        ],
       [0.09307671, 0.02135066, 0.02425244, ..., 0.01244815, 0.00110813,
        0.00080112]], dtype=float32)

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [20]:
clf = SVC(
    kernel = 'rbf',
    random_state = random_state,
    probability=True,
    C=0.5
)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_val)
score = accuracy_score(y_pred, y_val)
print("Accurancy: ",score)


Accurancy:  0.7295918367346939


In [21]:
y = list(clf.predict(X_train)) + list(clf.predict(X_val))
print(y.count(0))
print(y.count(1))
print(y.count(2))

44
76
92


In [22]:
input_img = cv2.imread("image_2.png")
normalized_img = preprocess_img(input_img)
y_pred = clf.predict([normalized_img])[0]
print(f'Normal prediction: {y_pred}')
y_pred_prob = clf.predict_proba([normalized_img])
prediction = np.argmax(y_pred_prob)
y_pred_prob = [f'{p:.10f}' for p in y_pred_prob[0]]
print(f'Probability: {y_pred_prob}')
print(f'Class with highest: {label_encoder.classes_[prediction]}') 

Normal prediction: 2
Probability: ['0.0018784531', '0.0779648657', '0.7393937737', '0.0634866046', '0.0065852374', '0.0016150551', '0.0732804840', '0.0072192909', '0.0285762355']
Class with highest: forbit


In [ ]:
with open("model2.pkl", "wb") as f:
    pickle.dump(clf, f)


In [5]:
input_img = cv2.imread("123.png")
normalized_img = preprocess_img(input_img)
type(normalized_img)

numpy.ndarray